# 4F13 - CW3 Code
Use a Jupyter Notebook as it takes so long

In [2]:
import scipy.io as sio
import numpy as np


data = sio.loadmat('kos_doc_data.mat')
A = np.array(data['A'])
B = data['B']
V = data['V']

## a - ML Multinomial

In [3]:
from tqdm import tqdm


W = np.max([np.max(A[:, 1]), np.max(B[:, 1])])  # total number of unique words
D = np.max(A[:, 0])  # number of documents in A

# Initialization: assign each document a mixture component at random
sw = np.zeros((W, 1))  # K multinomials over W unique words

# Populate the count matrices by looping over documents
print("Looping through documents to populate count matrices...")
for doc in tqdm(range(D)):
    doc_indices = np.where(A[:, 0] == doc+1)  # get all occurrences of document d in the training data
    word_ids = np.array(A[doc_indices, 1])  # unique word id's in document d
    counts = np.array(A[d_indices, 2])  # counts of words in document d
    sw[word_ids-1] += counts  # number of times w is assigned to component k


sorted_indices = np.argsort(counts)
print(sw[sorted_indices])

Looping through documents to populate count matrices...


NameError: name 'tqdm' is not defined